In [2]:
# ===== 0) Install (run once) =====
!pip -q install geopandas folium matplotlib-scalebar pyogrio shapely fiona openpyxl odfpy branca rtree pyproj



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.6/507.6 kB 34.1 MB/s eta 0:00:00


In [3]:
# ===== 1) Imports & global config =====
import os, io, re, json, time, unicodedata as ucn
from dataclasses import dataclass
from typing import Dict, Tuple, Optional, List

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrow
import folium
import branca.colormap as bcm

try:
    from matplotlib_scalebar.scalebar import ScaleBar
    HAS_SCALEBAR = True
except Exception:
    HAS_SCALEBAR = False

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 180)

# Colab Drive
from google.colab import drive
drive.mount("/content/drive")

# ---- Paths (edit DATA_DIR only) ----
DATA_DIR   = "/content/drive/MyDrive/data"   # << change if needed
VACANCY_CSV = os.path.join(DATA_DIR, "vacancy_municipality.csv")
GTFS_ZIP   = os.path.join(DATA_DIR, "gtfs_train.zip")
OUT_DIR    = "/content"
ARTIFACTS  = os.path.join(OUT_DIR, "artifacts")
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(ARTIFACTS, exist_ok=True)

# ---- Reproducibility ----
SEED = 42
np.random.seed(SEED)

# ---- CRS constants ----
CRS_WGS84 = "EPSG:4326"
CRS_CH    = "EPSG:2056"   # Swiss projected (meters)

# Small helper to persist quick PNGs
def _savefig(path):
    plt.tight_layout()
    plt.savefig(path, bbox_inches="tight")
    plt.close()


Mounted at /content/drive


In [4]:
# ===== 2) Helpers =====
def norm_name(s: str) -> str:
    """Normalize municipality names for joining."""
    if pd.isna(s): return s
    s = (str(s)
         .replace("ä","ae").replace("ö","oe").replace("ü","ue")
         .replace("Ä","ae").replace("Ö","oe").replace("Ü","ue")
         .replace("ß","ss"))
    s = ucn.normalize("NFKD", s).encode("ascii","ignore").decode("ascii")
    s = re.sub(r"[^A-Za-z0-9]+","", s).lower()
    return s

def read_bfs_like_csv(path: str) -> pd.DataFrame:
    """Read BFS SDMX-like CSV with semicolons; auto-detect header line."""
    text = None
    for enc in ("utf-8-sig","utf-8","latin1"):
        try:
            with open(path, "r", encoding=enc) as f:
                text = f.read()
            break
        except Exception:
            pass
    if text is None:
        raise IOError(f"Cannot read file {path} with utf-8/latin1.")
    lines = text.splitlines()
    hdr_idx, pat = 0, re.compile(r"(TIME_PERIOD|Zeitperiode).*(OBS_VALUE|Beobachtungswert)")
    for i, ln in enumerate(lines[:200]):
        if pat.search(ln): hdr_idx = i; break
    return pd.read_csv(io.StringIO("\n".join(lines[hdr_idx:])), sep=";", engine="python", dtype=str)

def parse_percent(x):
    """Parse text to decimal percent (e.g., '0.51' -> 0.51%)."""
    if x is None or (isinstance(x, float) and np.isnan(x)): return np.nan
    s = str(x).strip()
    if not s: return np.nan
    s = s.replace("\xa0"," ").replace("%","").strip()
    s = s.replace("’","").replace("'","").replace(" ", "").replace(",", ".")
    if re.search(r"\bjan\b", s, flags=re.I):  # Excel oddities (rare)
        nums = re.findall(r"(\d+)", s)
        return float(nums[0]) / 100.0 if nums else np.nan
    try:
        v = float(s)
    except:
        return np.nan
    return v/100.0 if v > 10 else v  # autoscale when 51 -> 0.51

def _key_str(x): return str(x).strip() if pd.notna(x) else x

def timer(msg: str):
    t0 = time.time()
    print(f"[start] {msg}")
    def _end():
        dt = time.time() - t0
        print(f"[done ] {msg} in {dt:.2f}s")
    return _end

def assert_cols(df, cols: List[str], name="df"):
    missing = [c for c in cols if c not in df.columns]
    assert not missing, f"{name} missing columns: {missing}"

def robust_range(series, qlo=1, qhi=99) -> Tuple[float,float]:
    s = pd.to_numeric(series, errors="coerce")
    s = s[np.isfinite(s)]
    if s.empty: return (0.0, 1.0)
    lo = float(np.nanpercentile(s, qlo))
    hi = float(np.nanpercentile(s, qhi))
    if not np.isfinite(lo) or not np.isfinite(hi) or hi <= lo:
        lo, hi = float(np.nanmin(s)), float(np.nanmax(s))
    if not np.isfinite(lo) or not np.isfinite(hi) or hi <= lo:
        lo, hi = 0.0, 1.0
    return lo, hi

def norm01(x, lo, hi):
    x = np.asarray(x, dtype="float64")
    lo, hi = float(lo), float(hi)
    if hi <= lo + 1e-9: return np.zeros_like(x)
    return (x - lo) / (hi - lo)

def exp_penalty_monotone(t_norm, k):
    """Convex penalty in [0,1]; larger k -> stronger penalty on long commutes."""
    t = np.clip(np.asarray(t_norm, dtype="float64"), 0.0, 1.0)
    k = float(k)
    if k <= 1e-6: return t
    return (1.0 - np.exp(-k * t)) / (1.0 - np.exp(-k))

def safe_to_crs(gdf, crs):
    g = gdf.copy()
    try:
        if g.crs is None:
            return g.set_crs(crs)
        elif str(g.crs).lower() != str(crs).lower():
            return g.to_crs(crs)
    except Exception:
        pass
    return g


In [5]:
# ===== 3) Swiss municipalities (latest geometries) =====
URL_GEM = ("https://data.opendatasoft.com/explore/dataset/"
           "georef-switzerland-gemeinde-millesime%40public/download/"
           "?format=geojson&timezone=Europe%2FBerlin")

t_end = timer("Load Swiss Gemeinde geometries")
gdf_all = gpd.read_file(URL_GEM)[["gem_name","gem_code","kan_code","year","geometry"]].rename(
    columns={"gem_name":"GEMEINDE_NAME","gem_code":"GEMEINDE_CODE","kan_code":"KANTON_CODE"}
)
latest_geom_year = gdf_all["year"].max()
gdf = (gdf_all[gdf_all["year"] == latest_geom_year]
       .sort_values(["GEMEINDE_CODE","year"])
       .drop_duplicates("GEMEINDE_CODE", keep="last")
       .copy())
gdf = safe_to_crs(gdf.set_geometry(gdf.geometry.buffer(0)), CRS_CH)
gdf["area_km2"] = gdf.area / 1e6
t_end()

print("Gemeinden:", len(gdf), "| year:", latest_geom_year)


[start] Load Swiss Gemeinde geometries


[done ] Load Swiss Gemeinde geometries in 66.81s
Gemeinden: 2128 | year: 2025


In [6]:
# ===== 4) BFS vacancy indicator (clean, join, QA) =====
t_end = timer("Load BFS vacancy indicator")
df_raw = read_bfs_like_csv(VACANCY_CSV)
t_end()

COL_GEM   = "Grossregionen, Kantone, Bezirke und Gemeinden"
COL_ROOMS = "Anzahl Zimmer"
COL_TYPE  = "Typ der leer stehenden Wohnung"
COL_MEAS  = "Art der Messung"
COL_VAL   = "OBS_VALUE" if "OBS_VALUE" in df_raw.columns else "Beobachtungswert"

df = df_raw.copy()
df["key_name"] = df[COL_GEM].astype(str).str.strip()

m_rooms = df[COL_ROOMS].astype(str).str.strip().str.lower().isin(["total","_t","gesamt"])
m_type  = df[COL_TYPE ].astype(str).str.lower().str.contains("alle", case=False)
m_meas  = df[COL_MEAS ].astype(str).str.lower().str.contains("anteil", case=False)
df = df[m_rooms & m_type & m_meas].copy()
df["vacancy_pct"] = df[COL_VAL].map(parse_percent)

NAME_MAP_RAW = {
    "Biel": "Biel/Bienne", "Bienne": "Biel/Bienne",
    "Leubringen": "Evilard",
    "Saint Imier": "Saint-Imier", "St Imier": "Saint-Imier", "St-Imier": "Saint-Imier",
    "Münster (BE)": "Moutier",
}
df["key_name_fix"] = df["key_name"].map(lambda s: NAME_MAP_RAW.get(str(s).strip(), s))

ind = (df.groupby("key_name_fix", as_index=False)["vacancy_pct"].mean()
         .rename(columns={"key_name_fix":"key_name"}))

gdf["key_norm"] = gdf["GEMEINDE_NAME"].map(norm_name)
ind["key_norm"] = ind["key_name"].map(norm_name)
g = gdf.merge(ind[["key_norm","vacancy_pct"]], on="key_norm", how="left")

# Drop known non-municipals
NON_MUNI_EXACT = {
    "Zürichsee (ZH)","Thunersee","Brienzersee","Bielersee (BE)","Bielersee (NE)",
    "Lac de Neuchâtel (BE)","Lac de Neuchâtel (NE)","Bodensee (SG)","Bodensee (TG)","Staatswald Galm"
}
mask_non = g["GEMEINDE_NAME"].isin(NON_MUNI_EXACT) | g["GEMEINDE_NAME"].str.contains(r"^Comunanza\b", case=False, na=False)
g = g.loc[~mask_non].copy()

print("Vacancy coverage:", g["vacancy_pct"].notna().mean().round(3), f"({g['vacancy_pct'].notna().sum()}/{len(g)})")


[start] Load BFS vacancy indicator
[done ] Load BFS vacancy indicator in 0.74s
Vacancy coverage: 0.995 (2106/2116)


In [7]:
# ===== 5) GTFS (all PT modes) → station graph with realism guards =====
import zipfile

if not os.path.exists("/content/gtfs"):
    with zipfile.ZipFile(GTFS_ZIP, "r") as z:
        z.extractall("/content/gtfs")

stops      = pd.read_csv("/content/gtfs/stops.txt")
trips      = pd.read_csv("/content/gtfs/trips.txt")
routes     = pd.read_csv("/content/gtfs/routes.txt")
stop_times = pd.read_csv("/content/gtfs/stop_times.txt")

# Use ALL modes (tram, metro, rail, bus, ferry, cable, gondola, funicular)
USE_ROUTE_TYPES = {0,1,2,3,4,5,6,7}
sel_routes = routes[routes["route_type"].isin(USE_ROUTE_TYPES)]["route_id"].unique()
sel_trips  = trips[trips["route_id"].isin(sel_routes)]["trip_id"].unique()
st_times   = stop_times[stop_times["trip_id"].isin(sel_trips)].copy()

def hms_to_sec(x: str) -> int:
    h, m, s = str(x).split(":")
    return int(h)*3600 + int(m)*60 + int(s)

# Map to station_id (parent_station fallback)
stops["station_id"] = stops["parent_station"].fillna(stops["stop_id"]).astype(str)
st_times["stop_id"] = st_times["stop_id"].astype(str)
st_times = st_times.merge(stops[["stop_id","station_id"]], on="stop_id", how="left")

st = st_times.dropna(subset=["arrival_time","departure_time","station_id"]).copy()
st["arr_s"] = st["arrival_time"].map(hms_to_sec)
st["dep_s"] = st["departure_time"].map(hms_to_sec)
st.sort_values(["trip_id","stop_sequence"], inplace=True)
st["next_station"] = st.groupby("trip_id")["station_id"].shift(-1)
st["next_arr_s"]   = st.groupby("trip_id")["arr_s"].shift(-1)

edges_raw = st.dropna(subset=["next_station","next_arr_s"])[["station_id","next_station","dep_s","next_arr_s"]].copy()
edges_raw["w_sec"] = (edges_raw["next_arr_s"] - edges_raw["dep_s"]).astype(float)
edge_min = (edges_raw.groupby(["station_id","next_station"], as_index=False)["w_sec"]
                     .min()
                     .rename(columns={"station_id":"u","next_station":"v"}))
edge_min["u"] = edge_min["u"].astype(str); edge_min["v"] = edge_min["v"].astype(str)

# Station points (EPSG:2056)
df_pts = stops[["station_id","stop_lon","stop_lat"]].dropna().drop_duplicates().copy()
df_pts["stop_lon"] = pd.to_numeric(df_pts["stop_lon"], errors="coerce")
df_pts["stop_lat"] = pd.to_numeric(df_pts["stop_lat"], errors="coerce")
df_pts = df_pts.dropna(subset=["stop_lon","stop_lat"]).reset_index(drop=True)

stations_g = gpd.GeoDataFrame(
    df_pts, geometry=gpd.points_from_xy(df_pts["stop_lon"], df_pts["stop_lat"]), crs=CRS_WGS84
).to_crs(CRS_CH)
stations_pts = stations_g.dissolve(by="station_id", as_index=False)
stations_pts["geometry"] = stations_pts.geometry.centroid
stxy = stations_pts.set_index("station_id")["geometry"].apply(lambda p: (p.x, p.y)).to_dict()

# Edge sanitization: min time 20s, max edge speed 160 km/h
def sanitize_edges(edges_df, stxy_dict, min_sec=20.0, vmax_kmh=160.0):
    u_xy = edges_df["u"].map(stxy_dict); v_xy = edges_df["v"].map(stxy_dict)
    dx = np.array([a[0]-b[0] if (a and b) else np.nan for a,b in zip(u_xy, v_xy)])
    dy = np.array([a[1]-b[1] if (a and b) else np.nan for a,b in zip(u_xy, v_xy)])
    dist_m = np.hypot(dx, dy)
    w = edges_df["w_sec"].astype(float).to_numpy()
    w = np.where(~np.isfinite(w) | (w < min_sec), min_sec, w)
    vmax_mps = vmax_kmh/3.6
    bad = (dist_m > 0) & (dist_m / w > vmax_mps)
    w[bad] = np.maximum(dist_m[bad] / vmax_mps, min_sec)
    out = edges_df.copy(); out["w_sec"] = w
    return out.dropna(subset=["w_sec"])

edge_sane = sanitize_edges(edge_min, stxy)

# Adjacency
from collections import defaultdict
adj = defaultdict(list)
for r in edge_sane.itertuples(index=False):
    adj[r.u].append((r.v, float(r.w_sec)))

print("Station graph:", len(edge_sane), "edges | stations:", len(stations_pts))


/tmp/ipython-input-4106543051.py:9: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  trips      = pd.read_csv("/content/gtfs/trips.txt")
/tmp/ipython-input-4106543051.py:10: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  routes     = pd.read_csv("/content/gtfs/routes.txt")
/tmp/ipython-input-4106543051.py:11: DtypeWarning: Columns (3,9) have mixed types. Specify dtype option on import or set low_memory=False.
  stop_times = pd.read_csv("/content/gtfs/stop_times.txt")


Station graph: 9526 edges | stations: 4771


In [8]:
# ===== REPLACE: compute_commute_minutes_from (fixed length-safe smoothing) =====
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Tunables (same semantics as before)
K_NEAREST_ST      = 5        # stations per Gemeinde for soft-min
WALK_KMPH         = 5.0
WALK_M_PER_MIN    = (WALK_KMPH * 1000) / 60.0
MAX_WALK_M        = 15000
SOFTMIN_TAU_MIN   = 8.0
SMOOTH_SIGMA_M    = 20000.0  # ~20 km smoothing kernel
VMAX_GLOBAL_KMH   = 160.0
BOARD_MIN         = 4.0

def _union_all_geoms(geoms):
    """Shapely 2.x: union_all; fallback to unary_union."""
    try:
        from shapely import union_all as _u
        return _u(list(geoms))
    except Exception:
        from shapely.ops import unary_union as _uu
        return _uu(list(geoms))

def _normtxt(s: str) -> str:
    return (str(s).lower()
            .replace("ä","a").replace("ö","o").replace("ü","u")
            .replace("é","e").replace("è","e").replace("ê","e").strip())

def match_station_ids_by_name(name: str):
    """Prefer exact normalized name, else prefix, else substring."""
    name_map = (stops.groupby("station_id")["stop_name"]
                      .agg(lambda s: s.value_counts().idxmax()))
    n0 = _normtxt(name)
    exact = [sid for sid, nm in name_map.items() if isinstance(nm,str) and _normtxt(nm)==n0]
    if exact: return sorted({str(x) for x in exact})
    starts = [sid for sid, nm in name_map.items() if isinstance(nm,str) and _normtxt(nm).startswith(n0)]
    if starts: return sorted({str(x) for x in starts})
    contains = [sid for sid, nm in name_map.items() if isinstance(nm,str) and n0 in _normtxt(nm)]
    return sorted({str(x) for x in contains})

def dijkstra_from_sources(src_ids):
    """Multi-source Dijkstra on the station graph. Requires global `adj`."""
    import heapq
    INF = 10**12
    dist = {sid: 0.0 for sid in src_ids}
    pq = [(0.0, sid) for sid in src_ids]
    heapq.heapify(pq)
    seen = set()
    while pq:
        d,u = heapq.heappop(pq)
        if u in seen:
            continue
        seen.add(u)
        for v,w in adj.get(u, []):
            nd = d + w
            if nd < dist.get(v, INF):
                dist[v] = nd
                heapq.heappush(pq, (nd, v))
    return pd.DataFrame({"station_id": list(dist.keys()), "time_sec_station": list(dist.values())})

def _ensure_lv95(gdf):
    """Ensure LV95 (EPSG:2056) for spatial math."""
    if gdf.crs is None:
        try:
            gdf = gdf.set_crs(2056)
        except Exception:
            pass
    if str(gdf.crs).lower() != "epsg:2056":
        gdf = gdf.to_crs(2056)
    return gdf

def compute_commute_minutes_from(origin_name: str, g_polys: gpd.GeoDataFrame) -> pd.DataFrame:
    """
    Compute per-Gemeinde minutes from origin_name with realism guards.
    Inputs:
      - origin_name: station name to match (e.g., 'Zürich HB')
      - g_polys: GeoDataFrame in LV95 with ['GEMEINDE_CODE','geometry'].
    Returns:
      - DataFrame ['GEMEINDE_CODE','avg_travel_min'] aligned 1:1 with the municipalities.
    """
    assert {"GEMEINDE_CODE","geometry"} <= set(g_polys.columns), \
        "g_polys must have ['GEMEINDE_CODE','geometry']"

    g_polys = _ensure_lv95(g_polys)

    seed_ids = match_station_ids_by_name(origin_name)
    if not seed_ids:
        raise ValueError(f"No station found for '{origin_name}'")

    # 1) Station-level shortest paths
    dist_df = dijkstra_from_sources(seed_ids)
    st_pts  = stations_pts.merge(dist_df, on="station_id", how="left")  # requires global stations_pts
    valid   = st_pts.dropna(subset=["time_sec_station"]).copy()
    if valid.empty:
        # keep output aligned to g_polys
        return g_polys[["GEMEINDE_CODE"]].assign(avg_travel_min=np.nan)

    # 2) In-polygon average (rail minutes)
    st_in = gpd.sjoin(
        valid[["station_id","time_sec_station","geometry"]],
        g_polys[["GEMEINDE_CODE","geometry"]],
        how="left", predicate="within"
    ).dropna(subset=["GEMEINDE_CODE"])
    inpoly = (st_in.groupby("GEMEINDE_CODE", as_index=False)["time_sec_station"].mean())
    inpoly["avg_travel_min"] = inpoly["time_sec_station"] / 60.0
    inpoly = inpoly[["GEMEINDE_CODE","avg_travel_min"]].copy()

    # 3) Soft-min via K nearest stations (rail + walk)
    g_cent = g_polys.copy()
    g_cent["centroid"] = g_cent.geometry.representative_point()
    g_cent = g_cent.set_geometry("centroid")

    # Build K-nearest efficiently with multiple sjoin_nearest passes (left length preserved)
    remaining = g_cent.copy()
    cand = valid[["station_id","time_sec_station","geometry"]].copy()
    nearest = []
    for k in range(K_NEAREST_ST):
        kn = gpd.sjoin_nearest(remaining, cand, how="left", distance_col=f"dist_m_{k}")
        # Normalize cols regardless of *_left/*_right suffixes
        if "GEMEINDE_CODE_left" in kn.columns:
            kn = kn.rename(columns={"GEMEINDE_CODE_left":"GEMEINDE_CODE"})
        if "time_sec_station_right" in kn.columns:
            kn = kn.rename(columns={"time_sec_station_right": f"time_sec_{k}"})
        elif "time_sec_station" in kn.columns:
            kn = kn.rename(columns={"time_sec_station": f"time_sec_{k}"})
        if "station_id_right" in kn.columns:
            kn = kn.rename(columns={"station_id_right": f"sid_{k}"})
        elif "station_id" in kn.columns:
            kn = kn.rename(columns={"station_id": f"sid_{k}"})
        nearest.append(kn[["GEMEINDE_CODE", f"sid_{k}", f"time_sec_{k}", f"dist_m_{k}"]].copy())

        # exclude matched to diversify
        matched = kn.get(f"sid_{k}", pd.Series(dtype=object)).dropna().astype(str).unique().tolist()
        if matched:
            cand = cand[~cand["station_id"].astype(str).isin(matched)].copy()

    knn = nearest[0]
    for tdf in nearest[1:]:
        knn = knn.merge(tdf, on="GEMEINDE_CODE", how="left")

    tot_cols=[]
    for k in range(K_NEAREST_ST):
        dcol, tcol = f"dist_m_{k}", f"time_sec_{k}"
        if dcol in knn.columns and tcol in knn.columns:
            knn[dcol] = np.minimum(knn[dcol].fillna(np.inf), MAX_WALK_M)
            walk_min  = knn[dcol] / WALK_M_PER_MIN
            rail_min  = knn[tcol] / 60.0
            knn[f"total_min_{k}"] = rail_min + walk_min
            tot_cols.append(f"total_min_{k}")

    if tot_cols:
        arr = knn[tot_cols].to_numpy(dtype="float64")                      # shape: [n_gemeinde, K]
        arr = np.where(np.isfinite(arr), arr, np.inf)
        Z   = np.exp(-np.clip(arr, 0, np.inf) / SOFTMIN_TAU_MIN)
        Z[arr == np.inf] = 0.0
        soft = -SOFTMIN_TAU_MIN * np.log(np.clip(Z.sum(axis=1), 1e-9, np.inf))
        knn_out = pd.DataFrame({"GEMEINDE_CODE": knn["GEMEINDE_CODE"].values, "softmin_min": soft})
    else:
        knn_out = pd.DataFrame({"GEMEINDE_CODE": knn["GEMEINDE_CODE"].values, "softmin_min": np.nan})

    # 4) Combine in-polygon & soft-min (rail+walk)
    comb = g_polys[["GEMEINDE_CODE","geometry"]].merge(inpoly, on="GEMEINDE_CODE", how="left")
    comb = comb.merge(knn_out, on="GEMEINDE_CODE", how="left")
    comb["avg_travel_min"] = comb["avg_travel_min"].fillna(comb["softmin_min"]).astype(float)
    comb = comb.drop(columns=["softmin_min"])

    # 5) Physics floor and origin=0 rule
    seeds_geom = stations_pts.loc[stations_pts["station_id"].isin(seed_ids), "geometry"]
    if len(seeds_geom) == 0:
        seeds_geom = stations_pts.iloc[:1]["geometry"]
    origin_pt = _union_all_geoms(seeds_geom).centroid

    rep_pts = comb["geometry"].representative_point()
    dist_m  = rep_pts.distance(origin_pt).to_numpy(dtype="float64")
    floor_min = BOARD_MIN + (dist_m / 1000.0) / VMAX_GLOBAL_KMH * 60.0
    comb["avg_travel_min"] = np.maximum(comb["avg_travel_min"].to_numpy(dtype="float64"), floor_min)

    origin_code = g_polys.loc[g_polys.contains(origin_pt), "GEMEINDE_CODE"]
    if not origin_code.empty:
        comb.loc[comb["GEMEINDE_CODE"] == origin_code.iloc[0], "avg_travel_min"] = 0.0

    # 6) Length-safe Gaussian smoothing across Gemeinden
    # Build arrays aligned to base rows (no joins that could duplicate)
    base = g_polys.copy()
    base["centroid"] = base.geometry.representative_point()
    base = base.set_geometry("centroid")

    src = base.merge(comb[["GEMEINDE_CODE","avg_travel_min"]], on="GEMEINDE_CODE", how="inner")
    src = src.dropna(subset=["avg_travel_min"])

    if len(src) == 0:
        return comb[["GEMEINDE_CODE","avg_travel_min"]]

    # Coordinates
    bx = base.geometry.x.to_numpy(dtype="float64")
    by = base.geometry.y.to_numpy(dtype="float64")
    sx = src.geometry.x.to_numpy(dtype="float64")
    sy = src.geometry.y.to_numpy(dtype="float64")
    st = src["avg_travel_min"].to_numpy(dtype="float64")

    # Distance matrix [n_base, n_src]
    dx = bx[:, None] - sx[None, :]
    dy = by[:, None] - sy[None, :]
    D  = np.hypot(dx, dy)

    # Gaussian weights and weighted mean per base row
    W  = np.exp(-0.5 * (D / SMOOTH_SIGMA_M)**2)
    num = W @ st
    den = W.sum(axis=1)
    smoothed = np.where(den > 0, num / den, np.nan)

    out = base[["GEMEINDE_CODE"]].copy()
    out["avg_travel_min"] = smoothed
    return out


In [9]:
# ===== 7) Compute commute from chosen origin (fixed end-to-end) =====
ORIGIN_STATION = "Zürich HB"  # change for exploration

# Work in LV95 for spatial ops and pass only required columns
g_m = g.to_crs(CRS_CH)[["GEMEINDE_CODE","geometry"]].copy()

# Compute minutes per Gemeinde from the selected origin
tt_from_origin = compute_commute_minutes_from(ORIGIN_STATION, g_m)

# Merge back (ensure keys are comparable strings)
tt_from_origin["GEMEINDE_CODE"] = tt_from_origin["GEMEINDE_CODE"].astype(str)
g["GEMEINDE_CODE"] = g["GEMEINDE_CODE"].astype(str)

# Replace old 'avg_travel_min' with the new one for this origin
g = g.drop(columns=[c for c in ["avg_travel_min"] if c in g.columns])
g = g.merge(tt_from_origin, on="GEMEINDE_CODE", how="left", validate="many_to_one")

g_tt_plot = g.dropna(subset=["avg_travel_min"]).copy()
print("Travel-time coverage:", len(g_tt_plot), "/", len(g))


Travel-time coverage: 2116 / 2116


In [10]:
# ===== 8) Preference scoring (canonical baseline + logistic from scenarios) =====
# Canonical score uses intercept + exponential penalty
CANON_A = 2.0     # housing weight
CANON_B = 2.0     # commute penalty weight
CANON_K = 3.0     # curvature strength

v_min, v_max = robust_range(g["vacancy_pct"])
t_min, t_max = robust_range(g["avg_travel_min"])

mask = g["vacancy_pct"].notna() & g["avg_travel_min"].notna()
g_pref = g.loc[mask, ["GEMEINDE_CODE","vacancy_pct","avg_travel_min"]].copy()

if len(g_pref):
    v_all = norm01(pd.to_numeric(g_pref["vacancy_pct"], errors="coerce").values, v_min, v_max)
    t_all = norm01(pd.to_numeric(g_pref["avg_travel_min"], errors="coerce").values, t_min, t_max)
    pen_t = exp_penalty_monotone(t_all, CANON_K)
    util0 = CANON_A * v_all - CANON_B * pen_t
    w0 = -np.median(util0)  # center near 50
    score = 100.0 * (1.0 / (1.0 + np.exp(-(w0 + util0))))
    g_pref["preference_score"] = score
    g = g.merge(g_pref[["GEMEINDE_CODE","preference_score"]], on="GEMEINDE_CODE", how="left")
else:
    g["preference_score"] = np.nan
    w0 = 0.0

print("Preference score coverage:", g["preference_score"].notna().mean().round(3))


Preference score coverage: 0.995


In [11]:
# ===== 9) (Optional) Static maps for notebook report =====
try:
    # Vacancy
    g_plot = g.dropna(subset=["vacancy_pct"]).copy()
    if len(g_plot):
        fig, ax = plt.subplots(figsize=(10,10), dpi=150)
        g_plot.plot(column="vacancy_pct", ax=ax, cmap="RdYlGn", edgecolor="white", linewidth=0.15,
                    legend=True, legend_kwds={"label":"Vacancy rate (%)","shrink":0.6})
        ax.set_axis_off(); ax.set_title("Vacancy rate", pad=10)
        if HAS_SCALEBAR: ax.add_artist(ScaleBar(dx=1, units="m", dimension="si-length", location="lower left", box_alpha=0.8))
        _savefig(os.path.join(OUT_DIR,"vacancy_gemeinden.png"))

    # Commute
    g_tt_plot = g.dropna(subset=["avg_travel_min"]).copy()
    if len(g_tt_plot):
        fig, ax = plt.subplots(figsize=(10,10), dpi=150)
        g_tt_plot.plot(column="avg_travel_min", ax=ax, cmap="RdYlGn_r", edgecolor="white", linewidth=0.15,
                       legend=True, legend_kwds={"label":"Commute time (min)","shrink":0.6})
        ax.set_axis_off(); ax.set_title("Commute time from origin", pad=10)
        if HAS_SCALEBAR: ax.add_artist(ScaleBar(dx=1, units="m", dimension="si-length", location="lower left", box_alpha=0.8))
        _savefig(os.path.join(OUT_DIR,"commute_from_origin.png"))

    # Preference score
    if g["preference_score"].notna().any():
        g_ps = g.dropna(subset=["preference_score"]).copy()
        fig, ax = plt.subplots(figsize=(10,10), dpi=150)
        g_ps.plot(column="preference_score", ax=ax, cmap="RdYlGn", edgecolor="white", linewidth=0.15,
                  legend=True, vmin=0, vmax=100, legend_kwds={"label":"Preference score (0–100)","shrink":0.6})
        ax.set_axis_off(); ax.set_title("Preference score", pad=10)
        if HAS_SCALEBAR: ax.add_artist(ScaleBar(dx=1, units="m", dimension="si-length", location="lower left", box_alpha=0.8))
        _savefig(os.path.join(OUT_DIR,"preference_score.png"))
    else:
        print("ℹ️ Preference score not available — skipping plot.")
except Exception as e:
    print(f"⚠️ Skipping static maps: {e}")



In [12]:
# ===== 10) Export artifacts (full + simplified + centroids + meta) =====
# Columns the app expects
export_cols = ["GEMEINDE_CODE","GEMEINDE_NAME","vacancy_pct","avg_travel_min","preference_score","geometry"]

g_export = g.copy()
if g_export.crs is None:
    try: g_export = g_export.set_crs(CRS_CH)
    except Exception: pass
try:
    g_export = g_export.to_crs(CRS_WGS84)
except Exception:
    pass

# Ensure required cols exist
for c in export_cols:
    if c not in g_export.columns and c != "geometry":
        g_export[c] = np.nan
g_export = g_export[export_cols]

# Full GeoJSON
gj_full = os.path.join(ARTIFACTS, "gemeinden.geojson")
g_export.to_file(gj_full, driver="GeoJSON")

# Simplified polygons (for faster app)
g_slim = g_export.copy()
g_slim["geometry"] = g_slim.geometry.simplify(0.0012, preserve_topology=True)  # ~120–200m
gj_slim = os.path.join(ARTIFACTS, "gemeinden_simplified.geojson")
g_slim.to_file(gj_slim, driver="GeoJSON")

# Centroids parquet (fast PyDeck mode)
cent = g_export.copy()
cent["geometry"] = cent.geometry.representative_point()
cent_pq = os.path.join(ARTIFACTS, "gemeinden_centroids.parquet")
cent.to_parquet(cent_pq, index=False)

# CSV (no geometry)
g_export.drop(columns=["geometry"]).to_csv(os.path.join(ARTIFACTS, "gemeinden.csv"), index=False)

# Stations lookup
station_choices = (stops[["station_id","stop_name"]].dropna().drop_duplicates().sort_values("stop_name"))
station_choices = station_choices[~station_choices["stop_name"].str.contains("Bahn-2000", case=False, na=False)]
station_choices.to_csv(os.path.join(ARTIFACTS, "stations_lookup.csv"), index=False)

# Meta
meta = {
    "origin_station": ORIGIN_STATION,
    "canonical_weights": {"a": float(CANON_A), "b": float(CANON_B), "w0": float(w0)},
    "penalty_k": float(CANON_K),
    "v_min": float(v_min), "v_max": float(v_max),
    "t_min": float(t_min), "t_max": float(t_max)
}
with open(os.path.join(ARTIFACTS, "meta.json"), "w") as f:
    json.dump(meta, f, indent=2)

print("✅ Exported to:", ARTIFACTS)
print("Files:", os.listdir(ARTIFACTS))


✅ Exported to: /content/artifacts
Files: ['gemeinden_centroids.parquet', 'stations_lookup.csv', 'meta.json', 'gemeinden_simplified.geojson', 'gemeinden.csv', 'gemeinden.geojson']


In [13]:
# ===== 10′b) Multi-origin export (top-20 busiest + 6 extras, origin=0 rule) =====
# Writes: tt_by_origin.parquet with [GEMEINDE_CODE, avg_travel_min, origin_name]
import math

# Build name map once
name_map = (stops.groupby("station_id")["stop_name"].agg(lambda s: s.value_counts().idxmax()))

def match_station_ids_by_name_exactish(nm: str) -> List[str]:
    n0 = _normtxt(nm)
    exact = [sid for sid, n in name_map.items() if isinstance(n, str) and _normtxt(n)==n0]
    if exact: return [str(exact[0])]
    starts = [sid for sid, n in name_map.items() if isinstance(n, str) and _normtxt(n).startswith(n0)]
    if starts: return [str(starts[0])]
    contains = [sid for sid, n in name_map.items() if isinstance(n, str) and n0 in _normtxt(n)]
    return [str(contains[0])] if contains else []

# Busiest stations
by_station = (st_times.groupby("station_id", as_index=False)["stop_id"]
                      .count().rename(columns={"stop_id":"events"})
                      .sort_values("events", ascending=False))
top20 = by_station.head(20).assign(origin_station_id=lambda d: d["station_id"].astype(str))
top20["origin_name"] = top20["origin_station_id"].map(name_map.to_dict())

# + 6 curated extras
EXTRA_ORIGINS = ["Neuchâtel","Fribourg/Freiburg","Sion","Lugano","Bellinzona","Chur"]
extra_rows=[]
for nm in EXTRA_ORIGINS:
    ids = match_station_ids_by_name_exactish(nm)
    if not ids:
        print(f"ℹ️ Extra origin '{nm}' not found; skip.")
        continue
    sid=ids[0]
    if sid in set(top20["origin_station_id"].astype(str)):
        continue
    extra_rows.append({"origin_station_id": sid, "origin_name": name_map.get(sid, nm)})
if extra_rows:
    top20 = pd.concat([top20[["origin_station_id","origin_name"]], pd.DataFrame(extra_rows)], ignore_index=True)

top20 = top20.dropna(subset=["origin_name"]).drop_duplicates("origin_station_id").reset_index(drop=True)
origin_names = sorted({str(x) for x in top20["origin_name"].tolist()}, key=str.casefold)
print("Origins:", ", ".join(origin_names))

# Compute per origin
rows=[]
for _, r in top20.iterrows():
    origin_name = str(r["origin_name"])
    res = compute_commute_minutes_from(origin_name, g_m[["GEMEINDE_CODE","geometry"]].copy())
    res["origin_name"] = origin_name
    rows.append(res[["GEMEINDE_CODE","avg_travel_min","origin_name"]])

tt_by_origin = pd.concat(rows, ignore_index=True) if rows else pd.DataFrame(columns=["GEMEINDE_CODE","avg_travel_min","origin_name"])
tt_by_origin["avg_travel_min"] = pd.to_numeric(tt_by_origin["avg_travel_min"], errors="coerce")

outp = os.path.join(ARTIFACTS, "tt_by_origin.parquet")
tt_by_origin.to_parquet(outp, index=False)
print(f"✅ Wrote {outp} ({len(tt_by_origin):,} rows, {tt_by_origin['origin_name'].nunique()} origins).")


Origins: Aarau, Bahn-2000-Strecke, Basel SBB, Bellinzona, Bern, Brig, Chur, Fribourg/Freiburg, Genève, Lausanne, Liestal, Lugano, Luzern, Morges, Neuchâtel, Olten, Renens VD, Sion, Spiez, St. Gallen, Thun, Winterthur, Zug, Zürich Flughafen, Zürich HB, Zürich Oerlikon
✅ Wrote /content/artifacts/tt_by_origin.parquet (55,016 rows, 26 origins).


In [14]:
# ===== 11) Lightweight QA checks & summary report =====
def _assert_exists(p):
    assert os.path.exists(p), f"Missing file: {p}"

for f in ["gemeinden.geojson","gemeinden_simplified.geojson","gemeinden_centroids.parquet","gemeinden.csv","stations_lookup.csv","meta.json"]:
    _assert_exists(os.path.join(ARTIFACTS, f))

gj = gpd.read_file(os.path.join(ARTIFACTS, "gemeinden.geojson"))
needed = ["GEMEINDE_CODE","GEMEINDE_NAME","vacancy_pct","avg_travel_min","preference_score","geometry"]
for c in needed:
    assert c in gj.columns, f"Column missing in geojson: {c}"

report = {
    "n_gemeinden": int(len(g)),
    "vacancy_coverage_pct": float(100 * g["vacancy_pct"].notna().mean()),
    "commute_coverage_pct": float(100 * g["avg_travel_min"].notna().mean()),
    "preference_coverage_pct": float(100 * g["preference_score"].notna().mean()),
    "vacancy_summary": g["vacancy_pct"].describe().to_dict(),
    "commute_summary": g["avg_travel_min"].describe().to_dict(),
}
with open(os.path.join(ARTIFACTS, "report.json"), "w") as f:
    json.dump(report, f, indent=2)
print("✅ Artifacts & report OK.")


✅ Artifacts & report OK.


In [15]:
ORIGIN_FOR_EXAMPLE = "Zürich HB"

# If many NaNs in avg_travel_min, try to recompute from Zürich HB (only if helper is available)
try:
    need_tt = ("avg_travel_min" not in g.columns) or (g["avg_travel_min"].isna().mean() > 0.1)
    if need_tt and "compute_commute_minutes_from" in globals():
        # Minimal LV95 frame for spatial join
        g_m = g.to_crs(CRS_CH)[["GEMEINDE_CODE","geometry"]].copy()
        tt_from_origin = compute_commute_minutes_from(ORIGIN_FOR_EXAMPLE, g_m)
        tt_from_origin["GEMEINDE_CODE"] = tt_from_origin["GEMEINDE_CODE"].astype(str)
        g["GEMEINDE_CODE"] = g["GEMEINDE_CODE"].astype(str)
        g = g.drop(columns=[c for c in ["avg_travel_min"] if c in g.columns])
        g = g.merge(tt_from_origin, on="GEMEINDE_CODE", how="left", validate="many_to_one")
        print(f"Recomputed commute times from '{ORIGIN_FOR_EXAMPLE}'. Coverage:",
              g["avg_travel_min"].notna().mean())
except Exception as e:
    print("Skipping recompute (helper not available or failed):", e)

# If preference_score missing, compute a canonical one using the same formula as Step 10
if "preference_score" not in g.columns or g["preference_score"].isna().all():
    import numpy as np, pandas as pd
    def norm01(x, lo, hi):
        x = np.asarray(pd.to_numeric(x, errors="coerce"), dtype="float64")
        return (x - lo) / (hi - lo + 1e-9)
    def exp_penalty_monotone(t_norm, k):
        t_norm = np.clip(np.asarray(t_norm, dtype="float64"), 0.0, 1.0)
        if k <= 1e-6: return t_norm
        return (1.0 - np.exp(-k * t_norm)) / (1.0 - np.exp(-k))

    # Use the same hyperparams exported in Step 10 (fallbacks if not present)
    CANON_A = 2.0
    CANON_B = 2.0
    CANON_K = 3.0

    v_min = float(np.nanmin(pd.to_numeric(g["vacancy_pct"], errors="coerce")))
    v_max = float(np.nanmax(pd.to_numeric(g["vacancy_pct"], errors="coerce")))
    t_min = float(np.nanmin(pd.to_numeric(g["avg_travel_min"], errors="coerce")))
    t_max = float(np.nanmax(pd.to_numeric(g["avg_travel_min"], errors="coerce")))

    mask = g["vacancy_pct"].notna() & g["avg_travel_min"].notna()
    v_all = norm01(g.loc[mask, "vacancy_pct"],    v_min, v_max)
    t_all = norm01(g.loc[mask, "avg_travel_min"], t_min, t_max)
    util0 = CANON_A * v_all - CANON_B * exp_penalty_monotone(t_all, CANON_K)

    # Center around 50
    w0 = -np.median(util0)
    score = 100.0 * (1.0 / (1.0 + np.exp(-(w0 + util0))))
    g.loc[mask, "preference_score"] = np.clip(score, 0, 100)

    print("Computed canonical preference_score for example plots.")


In [16]:
# ===== Notebook Folium maps (origin=0 + zero-shift) =====
# Place this AFTER your export step (section 10), so artifacts exist.

import os, json, numpy as np, pandas as pd, geopandas as gpd
import folium, branca.colormap as bcm
from pathlib import Path

ARTIFACTS = Path("/content/artifacts")
GEOJSON   = ARTIFACTS / "gemeinden.geojson"
META_JSON = ARTIFACTS / "meta.json"
TTO_PARQ  = ARTIFACTS / "tt_by_origin.parquet"   # optional (multi-origin)

assert GEOJSON.exists() and META_JSON.exists(), "Run the export step first (section 10)."

# --- Load data ---
g = gpd.read_file(GEOJSON)
if g.crs is None or str(g.crs).lower() != "epsg:4326":
    g = g.to_crs(4326)

meta = json.loads(META_JSON.read_text(encoding="utf-8"))

tto = None
if TTO_PARQ.exists():
    tto = pd.read_parquet(TTO_PARQ)
    tto["GEMEINDE_CODE"]  = tto["GEMEINDE_CODE"].astype(str)
    tto["avg_travel_min"] = pd.to_numeric(tto["avg_travel_min"], errors="coerce")

# --- Helpers ---
def robust_range(series, lo_q=1, hi_q=99):
    s = pd.to_numeric(series, errors="coerce")
    s = s[np.isfinite(s)]
    if s.empty: return (0.0, 1.0)
    lo = float(np.nanpercentile(s, lo_q))
    hi = float(np.nanpercentile(s, hi_q))
    if not np.isfinite(lo) or not np.isfinite(hi) or hi <= lo:
        lo, hi = float(np.nanmin(s)), float(np.nanmax(s))
    if not np.isfinite(lo) or not np.isfinite(hi) or hi <= lo:
        lo, hi = 0.0, 1.0
    return lo, hi

def norm01(x, lo, hi):
    x = np.asarray(x, dtype="float64")
    if hi <= lo + 1e-9: return np.zeros_like(x)
    return (np.clip(x, lo, hi) - lo) / (hi - lo)

def commute_penalty_shape(t_norm, k):
    # Smooth, monotone penalty; larger k ≈ stronger curvature
    t = np.clip(np.asarray(t_norm, dtype="float64"), 0.0, 1.0)
    kk = max(0.2, float(k))
    return 1.0 - np.power(1.0 - t, kk)

def enforce_origin_zero_and_shift(df_wgs84: gpd.GeoDataFrame,
                                  tt_sel: pd.DataFrame | None) -> gpd.GeoDataFrame:
    """
    If per-origin times are provided, merge them and:
      1) force the min-time municipality(ies) to 0.0,
      2) subtract the (per-origin) global min from all effective times so tooltips show minutes from origin.
    If not provided, zero-shift the existing avg_travel_min in df.
    """
    df = df_wgs84.copy()
    df["GEMEINDE_CODE"] = df["GEMEINDE_CODE"].astype(str)

    if tt_sel is not None and not tt_sel.empty:
        tmp = tt_sel[["GEMEINDE_CODE","avg_travel_min"]].copy()
        tmp = tmp.rename(columns={"avg_travel_min": "avg_travel_min_origin"})
        tmp["GEMEINDE_CODE"] = tmp["GEMEINDE_CODE"].astype(str)
        df = df.merge(tmp, on="GEMEINDE_CODE", how="left")
        df["avg_travel_min_eff"] = df["avg_travel_min_origin"].combine_first(df.get("avg_travel_min"))
    else:
        df["avg_travel_min_eff"] = df.get("avg_travel_min")

    # Zero-shift by the minimum and clamp at 0
    cur = pd.to_numeric(df["avg_travel_min_eff"], errors="coerce")
    if cur.notna().any():
        mmin = float(np.nanmin(cur))
        # set the municipality(ies) achieving the min to exactly 0.0
        min_mask = (cur == mmin)
        df.loc[min_mask, "avg_travel_min_eff"] = 0.0
        # subtract min for everyone to show minutes-from-origin in tooltips
        df.loc[~min_mask, "avg_travel_min_eff"] = (cur - mmin)[~min_mask]
    df["avg_travel_min_eff"] = pd.to_numeric(df["avg_travel_min_eff"], errors="coerce").clip(lower=0)

    return df

def folium_choropleth(df_map, value_col, legend_label, colors, vmin=None, vmax=None, tooltip_cols=()):
    df_map = df_map.copy()
    try:
        df_map["geometry"] = df_map.geometry.simplify(0.0005, preserve_topology=True)
    except Exception:
        pass

    vals = pd.to_numeric(df_map[value_col], errors="coerce")
    if vmin is None: vmin = float(np.nanpercentile(vals, 1)) if vals.notna().any() else 0.0
    if vmax is None: vmax = float(np.nanpercentile(vals, 99)) if vals.notna().any() else 1.0

    cmap = bcm.LinearColormap(colors=colors, vmin=vmin, vmax=vmax); cmap.caption = legend_label
    m = folium.Map(location=[46.8, 8.2], zoom_start=7, tiles="OpenStreetMap")

    # Friendly tooltips
    fields, aliases = [], []
    for col, alias in tooltip_cols:
        fields.append(col); aliases.append(alias)

    def style_fn(feat):
        try: val = float(feat["properties"].get(value_col, None))
        except Exception: val = None
        fill = cmap(val) if val is not None else "#cccccc"
        return {"fillColor": fill, "color": "white", "weight": 0.2, "fillOpacity": 0.85}

    folium.GeoJson(
        data=df_map.to_json(),
        style_function=style_fn,
        tooltip=folium.features.GeoJsonTooltip(fields=fields, aliases=aliases, localize=True),
    ).add_to(m)
    cmap.add_to(m)
    display(m)

# --- Set origin and build effective times (origin=0 + zero-shift) ---
ORIGIN_NAME = "Zürich HB"

if tto is not None and "origin_name" in tto.columns and ORIGIN_NAME in tto["origin_name"].astype(str).unique():
    tt_sel = tto.loc[tto["origin_name"] == ORIGIN_NAME].copy()
else:
    tt_sel = None  # fall back to whatever is already in g

g_eff = enforce_origin_zero_and_shift(g, tt_sel)

# --- Compute preference score (using same mechanics as app) ---
cw = meta.get("canonical_weights", {})
w0 = float(cw.get("w0", 0.0)); a = float(cw.get("a", 2.0)); b = float(cw.get("b", 2.0))
k  = float(meta.get("penalty_k", 1.5))

v_lo, v_hi = robust_range(g_eff["vacancy_pct"])
t_lo, t_hi = robust_range(g_eff["avg_travel_min_eff"])

v_all = norm01(pd.to_numeric(g_eff["vacancy_pct"], errors="coerce").values, v_lo, v_hi)
t_all = norm01(pd.to_numeric(g_eff["avg_travel_min_eff"], errors="coerce").values, t_lo, t_hi)
pen_t = commute_penalty_shape(t_all, k)
util  = (w0 + a * v_all - b * pen_t)

finite = np.isfinite(util)
if finite.any():
    util = util - np.median(util[finite])

g_eff["preference_score"] = 100.0 * (1.0 / (1.0 + np.exp(-util)))

# --- Render 3 Folium maps ---
# 1) Commute only (now origin municipality shows 0.0 in tooltip)
g1 = g_eff.dropna(subset=["avg_travel_min_eff"]).copy()
g1["time_round"] = pd.to_numeric(g1["avg_travel_min_eff"], errors="coerce").round(1)
vmin_c = 0.0
vmax_c = float(np.nanpercentile(g1["avg_travel_min_eff"], 99)) if g1["avg_travel_min_eff"].notna().any() else 120.0
folium_choropleth(
    g1, "avg_travel_min_eff",
    legend_label=f"Commute time from {ORIGIN_NAME} (min)",
    colors=["green","yellow","red"],
    vmin=vmin_c, vmax=vmax_c,
    tooltip_cols=[("GEMEINDE_NAME","Municipality"), ("time_round","Commute (min)")]
)





Output hidden; open in https://colab.research.google.com to view.

In [17]:
# 2) Housing only
g2 = g_eff.dropna(subset=["vacancy_pct"]).copy()
g2["vacancy_round"] = pd.to_numeric(g2["vacancy_pct"], errors="coerce").round(2)
folium_choropleth(
    g2, "vacancy_pct",
    legend_label="Vacancy rate (%)",
    colors=["red","yellow","green"],
    tooltip_cols=[("GEMEINDE_NAME","Municipality"), ("vacancy_round","Vacancy (%)")]
)



Output hidden; open in https://colab.research.google.com to view.

In [18]:
# 3) Preference score
g3 = g_eff.dropna(subset=["preference_score"]).copy()
g3["score_round"] = pd.to_numeric(g3["preference_score"], errors="coerce").round(1)
g3["time_round"]  = pd.to_numeric(g3["avg_travel_min_eff"], errors="coerce").round(1)
g3["vacancy_round"] = pd.to_numeric(g3["vacancy_pct"], errors="coerce").round(2)
folium_choropleth(
    g3, "preference_score",
    legend_label="Preference score (0–100)",
    colors=["red","yellow","green"],
    vmin=0.0, vmax=100.0,
    tooltip_cols=[("GEMEINDE_NAME","Municipality"),
                  ("score_round","Score"),
                  ("vacancy_round","Vacancy (%)"),
                  ("time_round","Commute (min)")]
)

Output hidden; open in https://colab.research.google.com to view.

In [19]:
# ===== 12) Environment metadata (for reproducibility) =====
import platform, sys, pkgutil
meta_env = {
    "python_version": sys.version,
    "platform": platform.platform(),
    "notebook_seed": SEED,
    "packages": {m.name: None for m in pkgutil.iter_modules()}  # lightweight presence list
}
with open(os.path.join(ARTIFACTS, "env_info.json"), "w") as f:
    json.dump(meta_env, f, indent=2)
print("Saved env_info.json")


Saved env_info.json


In [20]:
# --- Overwrite app.py (robust scenarios + origin=0 + zero-shift) ---
import os, shutil
APP_DIR  = "/content/streamlit_app"
DATA_DIR = os.path.join(APP_DIR, "data")
APP_PATH = os.path.join(APP_DIR, "app.py")
os.makedirs(DATA_DIR, exist_ok=True)

# Ensure artifacts exist locally
shutil.copyfile("/content/artifacts/gemeinden.geojson", f"{DATA_DIR}/gemeinden.geojson")
shutil.copyfile("/content/artifacts/meta.json",       f"{DATA_DIR}/meta.json")
if os.path.exists("/content/artifacts/tt_by_origin.parquet"):
    shutil.copyfile("/content/artifacts/tt_by_origin.parquet", f"{DATA_DIR}/tt_by_origin.parquet")

app_code = r'''
import os, json, hashlib
import numpy as np
import pandas as pd
import geopandas as gpd
import streamlit as st
from pathlib import Path

# ---------- Setup ----------
try:
    BASE_DIR = Path(__file__).resolve().parent
except NameError:
    BASE_DIR = Path.cwd()
DATA_DIR  = BASE_DIR / "data"

st.set_page_config(page_title="Swiss Commute & Housing Explorer", layout="wide")

# ---------- Utils ----------
def robust_range(series_like, lo_q=1, hi_q=99):
    s = pd.to_numeric(pd.Series(series_like).astype("float64"), errors="coerce")
    s = s[np.isfinite(s)]
    if s.empty:
        return (0.0, 1.0)
    lo = float(np.nanpercentile(s, lo_q))
    hi = float(np.nanpercentile(s, hi_q))
    if not np.isfinite(lo) or not np.isfinite(hi) or hi <= lo:
        lo, hi = float(np.nanmin(s)), float(np.nanmax(s))
    if not np.isfinite(lo) or not np.isfinite(hi) or hi <= lo:
        lo, hi = 0.0, 1.0
    return lo, hi

def norm01_clipped(x, lo, hi):
    x = np.asarray(x, dtype="float64")
    lo, hi = float(lo), float(hi)
    if hi <= lo + 1e-9:
        return np.zeros_like(x)
    x = np.clip(x, lo, hi)
    return (x - lo) / (hi - lo)

def commute_penalty_shape(t_norm, k):
    # Smooth monotone penalty (k controls curvature)
    t = np.clip(np.asarray(t_norm, dtype="float64"), 0.0, 1.0)
    kk = max(0.2, float(k))
    return 1.0 - np.power(1.0 - t, kk)

def enforce_origin_zero_and_shift(df_wgs84: gpd.GeoDataFrame,
                                  tt_sel: pd.DataFrame | None) -> gpd.GeoDataFrame:
    """
    If per-origin times are provided, merge them:
      - put into avg_travel_min_eff
      - force min municipality(ies) to 0.0
      - subtract global min so tooltips show minutes-from-origin
    Else, do the same on existing avg_travel_min.
    """
    df = df_wgs84.copy()
    df["GEMEINDE_CODE"] = df["GEMEINDE_CODE"].astype(str)

    if tt_sel is not None and not tt_sel.empty:
        tmp = tt_sel[["GEMEINDE_CODE","avg_travel_min"]].copy()
        tmp = tmp.rename(columns={"avg_travel_min": "avg_travel_min_origin"})
        tmp["GEMEINDE_CODE"] = tmp["GEMEINDE_CODE"].astype(str)
        df = df.merge(tmp, on="GEMEINDE_CODE", how="left")
        df["avg_travel_min_eff"] = df["avg_travel_min_origin"].combine_first(df.get("avg_travel_min"))
    else:
        df["avg_travel_min_eff"] = df.get("avg_travel_min")

    cur = pd.to_numeric(df["avg_travel_min_eff"], errors="coerce")
    if cur.notna().any():
        mmin = float(np.nanmin(cur))
        min_mask = (cur == mmin)
        df.loc[min_mask, "avg_travel_min_eff"] = 0.0
        df.loc[~min_mask, "avg_travel_min_eff"] = (cur - mmin)[~min_mask]
    df["avg_travel_min_eff"] = pd.to_numeric(df["avg_travel_min_eff"], errors="coerce").clip(lower=0)
    return df

def resolve_prior(meta, session):
    if "manual_weights" in session:
        mw = session["manual_weights"]
        return (float(mw.get("w0", 0.0)),
                float(max(0.0, mw.get("w_v", 2.0))),
                float(max(0.0, mw.get("w_t", 2.0))))
    try:
        cw = meta["canonical_weights"]
        return (float(cw.get("w0", 0.0)),
                float(max(0.0, cw.get("a", 2.0))),
                float(max(0.0, cw.get("b", 2.0))))
    except Exception:
        return (0.0, 2.0, 2.0)

def answers_signature(choices_dict):
    key_order = sorted(choices_dict.keys())
    s = "|".join(f"{k}:{choices_dict[k]}" for k in key_order)
    return hashlib.sha256(s.encode("utf-8")).hexdigest()

def build_anchor_pairs(v_range, t_range, n_each=3, dv_n=0.18, dt_n=0.10):
    v_lo, v_hi = v_range; t_lo, t_hi = t_range
    v_mid = (v_lo + v_hi) / 2.0
    t_mid = (t_lo + t_hi) / 2.0
    dv = dv_n * (v_hi - v_lo)
    dt = dt_n * (t_hi - t_lo)
    rows = []
    for _ in range(n_each):
        rows.append({"A_vacancy_pct": v_mid, "A_travel_min": t_mid - dt,
                     "B_vacancy_pct": v_mid, "B_travel_min": t_mid + dt,
                     "choice": "A", "w": 0.15})
        rows.append({"A_vacancy_pct": v_mid + dv, "A_travel_min": t_mid,
                     "B_vacancy_pct": v_mid - dv, "B_travel_min": t_mid,
                     "choice": "A", "w": 0.15})
    return pd.DataFrame(rows)

def fit_logistic_weighted(sc_df, v_range, t_range, k_ref,
                          prior=(0.0,2.0,2.0), l2=0.25,
                          clip_w0=(-1.5,1.5), clip_a=(0.0,4.0), clip_b=(0.0,4.0),
                          iters=800, lr=0.25):
    v_lo, v_hi = v_range; t_lo, t_hi = t_range
    vA = sc_df["A_vacancy_pct"].values; vB = sc_df["B_vacancy_pct"].values
    tA = sc_df["A_travel_min"].values;  tB = sc_df["B_travel_min"].values
    y  = (sc_df["choice"].values == "A").astype(float)
    w_samp = sc_df.get("w", pd.Series(1.0, index=sc_df.index)).values.astype("float64")
    Va = norm01_clipped(vA, v_lo, v_hi); Vb = norm01_clipped(vB, v_lo, v_hi)
    Ta = norm01_clipped(tA, t_lo, t_hi); Tb = norm01_clipped(tB, t_lo, t_hi)
    Pa = commute_penalty_shape(Ta, k_ref); Pb = commute_penalty_shape(Tb, k_ref)
    X = np.column_stack([np.ones(len(sc_df)), (Va - Vb), -(Pa - Pb)])  # [w0,a,b]
    w = np.array(prior, dtype="float64")
    denom = max(1.0, np.sum(w_samp))
    for _ in range(iters):
        z = X @ w
        p = 1.0 / (1.0 + np.exp(-z))
        grad_ll = X.T @ ((y - p) * w_samp) / denom
        grad_prior = -l2 * (w - np.array(prior))
        w += lr * (grad_ll + grad_prior)
        w[0] = np.clip(w[0], *clip_w0); w[1] = np.clip(w[1], *clip_a); w[2] = np.clip(w[2], *clip_b)
    w[1] = max(0.0, w[1]); w[2] = max(0.0, w[2])
    return tuple(w)

@st.cache_data(show_spinner=False)
def build_edge_tradeoffs(_df, v_range, t_range, n=10, seed=42,
                         dtnorm_range=(0.05, 0.18), dvnorm_range=(0.10, 0.30)):
    """
    Robust scenario builder: always returns a DataFrame with
    [qid, A_gem, A_vacancy_pct, A_travel_min, B_gem, B_vacancy_pct, B_travel_min]
    where A and B are hard trade-offs (one better housing, the other better commute).
    """
    # Coerce to DataFrame safely
    if isinstance(_df, (pd.DataFrame, gpd.GeoDataFrame)):
        df = _df.copy()
    else:
        df = pd.DataFrame(_df)

    # Ensure necessary columns exist
    need = ["GEMEINDE_NAME","vacancy_pct","avg_travel_min"]
    for c in need:
        if c not in df.columns:
            df[c] = np.nan

    df = df[need].copy()
    df["vacancy_pct"]    = pd.to_numeric(df["vacancy_pct"], errors="coerce")
    df["avg_travel_min"] = pd.to_numeric(df["avg_travel_min"], errors="coerce")
    df = df.dropna(subset=["GEMEINDE_NAME","vacancy_pct","avg_travel_min"])

    if df.empty:
        return pd.DataFrame(columns=["qid","A_gem","A_vacancy_pct","A_travel_min","B_gem","B_vacancy_pct","B_travel_min"])

    v_lo, v_hi = v_range; t_lo, t_hi = t_range
    df["v_n"] = norm01_clipped(df["vacancy_pct"].values, v_lo, v_hi)
    df["t_n"] = norm01_clipped(df["avg_travel_min"].values, t_lo, t_hi)

    rng = np.random.default_rng(seed)
    used = set(); rows = []; tries = 0
    while len(rows) < n and tries < 8000:
        tries += 1
        A = df.sample(1, random_state=int(rng.integers(1, 1_000_000))).iloc[0]
        if A["GEMEINDE_NAME"] in used:
            continue
        orientation = int(rng.integers(0, 2))

        if orientation == 0:
            # A better housing, B better commute
            cand = df[
                (df["v_n"] < A["v_n"]) &
                (df["t_n"] < A["t_n"]) &
                ((A["v_n"] - df["v_n"]).between(dvnorm_range[0], dvnorm_range[1], inclusive="both")) &
                ((A["t_n"] - df["t_n"]).between(dtnorm_range[0], dtnorm_range[1], inclusive="both")) &
                (~df["GEMEINDE_NAME"].isin(used | {A["GEMEINDE_NAME"]}))
            ]
        else:
            # A better commute, B better housing
            cand = df[
                (df["v_n"] > A["v_n"]) &
                (df["t_n"] > A["t_n"]) &
                ((df["v_n"] - A["v_n"]).between(dvnorm_range[0], dvnorm_range[1], inclusive="both")) &
                ((df["t_n"] - A["t_n"]).between(dtnorm_range[0], dtnorm_range[1], inclusive="both")) &
                (~df["GEMEINDE_NAME"].isin(used | {A["GEMEINDE_NAME"]}))
            ]

        if cand.empty:
            continue
        B = cand.sample(1, random_state=int(rng.integers(1, 1_000_000))).iloc[0]
        rows.append({
            "qid": len(rows)+1,
            "A_gem": A["GEMEINDE_NAME"], "A_vacancy_pct": float(A["vacancy_pct"]), "A_travel_min": float(A["avg_travel_min"]),
            "B_gem": B["GEMEINDE_NAME"], "B_vacancy_pct": float(B["vacancy_pct"]), "B_travel_min": float(B["avg_travel_min"]),
        })
        used.add(A["GEMEINDE_NAME"]); used.add(B["GEMEINDE_NAME"])

    return pd.DataFrame(rows)

@st.cache_data(show_spinner=False)
def load_data():
    gj = DATA_DIR / "gemeinden.geojson"
    mj = DATA_DIR / "meta.json"
    if not gj.exists() or not mj.exists():
        st.error("Artifacts not found. Re-run the export step in the notebook.")
        st.stop()
    g = gpd.read_file(str(gj))
    if g.crs is None or str(g.crs).lower() != "epsg:4326":
        g = g.to_crs(4326)
    for c in ["vacancy_pct","avg_travel_min","preference_score"]:
        if c in g.columns:
            g[c] = pd.to_numeric(g[c], errors="coerce")
    meta = json.loads((mj).read_text(encoding="utf-8"))

    tto = None
    ttp = DATA_DIR / "tt_by_origin.parquet"
    if ttp.exists():
        try:
            tto = pd.read_parquet(ttp)
            tto["GEMEINDE_CODE"]  = tto["GEMEINDE_CODE"].astype(str)
            tto["avg_travel_min"] = pd.to_numeric(tto["avg_travel_min"], errors="coerce")
            if "origin_name" not in tto.columns and "origin_station_id" in tto.columns:
                tto["origin_name"] = tto["origin_station_id"].astype(str)
        except Exception:
            tto = None
    return g, meta, tto

# ---------- App ----------
g, meta, tt_by_origin = load_data()

st.title("Swiss Commute & Housing Explorer (Notebook Demo)")
st.markdown(
    "Three modes: **Preference score**, **Housing only**, **Commute only**. "
    "Pick the **origin SBB station** in the **left sidebar**. "
    "The Preference score blends **vacancy** (higher is better) and **commute** (shorter is better)."
)

# ---------- Sidebar controls ----------
with st.sidebar:
    st.header("Controls")
    def _is_bahn2000(name: str) -> bool:
        s = str(name).lower()
        return ("bahn" in s) and ("2000" in s)

    if tt_by_origin is not None and "origin_name" in tt_by_origin.columns:
        stations_raw = tt_by_origin["origin_name"].dropna().astype(str).unique().tolist()
        stations = sorted([s for s in stations_raw if not _is_bahn2000(s)], key=str.casefold)
        default_origin = meta.get("origin_station", "Zürich HB")
        idx = stations.index(default_origin) if default_origin in stations else 0
        origin_name = st.selectbox("Origin SBB station", options=stations, index=idx)
    else:
        origin_name = meta.get("origin_station", "Zürich HB")
        st.info(f"Dynamic origins not found; using precomputed origin: **{origin_name}**")

    map_mode = st.radio("Map mode", ["Preference score","Housing only","Commute only"])
    penalty_k = st.slider(
        "Commute penalty curvature (k)",
        0.2, 6.0, float(meta.get("penalty_k", 1.5)), 0.1,
        help="Adjusts the curvature used in Preference score."
    )

# ---------- Effective commute times for selected origin (origin=0 + zero-shift) ----------
df = g.copy()
df["GEMEINDE_CODE"] = df["GEMEINDE_CODE"].astype(str)

if tt_by_origin is not None and origin_name in tt_by_origin.get("origin_name", pd.Series(dtype=str)).astype(str).unique():
    tt_sel = tt_by_origin.loc[tt_by_origin["origin_name"] == origin_name, ["GEMEINDE_CODE","avg_travel_min"]].copy()
else:
    tt_sel = None

df = enforce_origin_zero_and_shift(df, tt_sel)

# Simplify for speed
try:
    df["geometry"] = df.geometry.simplify(0.0005, preserve_topology=True)
except Exception:
    pass

# ---------- Robust ranges from *effective* data ----------
v_range = robust_range(df["vacancy_pct"])
t_range = robust_range(df["avg_travel_min_eff"])

# ---------- Build scenarios (edge-case tradeoffs) ----------
df_scen = df[["GEMEINDE_NAME","vacancy_pct","avg_travel_min_eff"]].copy()
df_scen = df_scen.rename(columns={"avg_travel_min_eff":"avg_travel_min"})
sc = build_edge_tradeoffs(
    df_scen, v_range=v_range, t_range=t_range, n=10, seed=42,
    dtnorm_range=(0.05, 0.18), dvnorm_range=(0.10, 0.30)
)

# ---------- Tabs ----------
tab_map, tab_pref, tab_data = st.tabs(["🗺️ Map","🧭 Preference elicitation","📄 Data preview"])

# ---------- Preference elicitation (live; no button) ----------
with tab_pref:
    st.subheader("10 trade-off questions (hard choices)")
    if sc.empty:
        st.warning("Not enough data to build scenarios.")
    else:
        for _, row in sc.iterrows():
            st.markdown("---")
            c1, c2, c3 = st.columns([1,1,1])
            with c1:
                st.markdown(f"**Q{int(row.qid)} – Option A**")
                st.write(row.A_gem)
                st.write(f"Vacancy: {row.A_vacancy_pct:.2f}%")
                st.write(f"Commute: {row.A_travel_min:.1f} min")
            with c2:
                st.markdown(f"**Q{int(row.qid)} – Option B**")
                st.write(row.B_gem)
                st.write(f"Vacancy: {row.B_vacancy_pct:.2f}%")
                st.write(f"Commute: {row.B_travel_min:.1f} min")
            with c3:
                st.radio(
                    f"Your choice for Q{int(row.qid)}",
                    ["A","B"], key=f"choice_{int(row.qid)}",
                    horizontal=True, index=None
                )

# ---------- Learn weights only when answers change (k_ref fixed) ----------
def answers_signature(choices_dict):
    key_order = sorted(choices_dict.keys())
    s = "|".join(f"{k}:{choices_dict[k]}" for k in key_order)
    return hashlib.sha256(s.encode("utf-8")).hexdigest()

choices_dict, answered_rows = {}, []
for _, r in sc.iterrows():
    key = f"choice_{int(r.qid)}"
    ch = st.session_state.get(key, None)
    if ch in ("A","B"):
        choices_dict[key] = ch
        answered_rows.append(r)

ans_sig = answers_signature(choices_dict) if choices_dict else None

if answered_rows:
    user_sc = pd.DataFrame(answered_rows).reset_index(drop=True).copy()
    user_sc["choice"] = [choices_dict[f"choice_{int(r.qid)}"] for _, r in user_sc.iterrows()]
    user_sc["w"] = 1.0
    anchors = build_anchor_pairs(v_range, t_range, n_each=3, dv_n=0.18, dt_n=0.10)
    train_df = pd.concat([
        user_sc[["A_vacancy_pct","A_travel_min","B_vacancy_pct","B_travel_min","choice","w"]],
        anchors[["A_vacancy_pct","A_travel_min","B_vacancy_pct","B_travel_min","choice","w"]],
    ], ignore_index=True)
else:
    train_df = None

if train_df is not None and st.session_state.get("answers_sig") != ans_sig:
    prior = resolve_prior(meta, st.session_state)
    try:
        w0, a, b = fit_logistic_weighted(
            train_df, v_range=v_range, t_range=t_range, k_ref=1.0,
            prior=prior, l2=0.25, iters=900, lr=0.25
        )
        st.session_state["manual_weights"] = {
            "w0": float(w0), "w_v": float(a), "w_t": float(b),
            "v_min": float(v_range[0]), "v_max": float(v_range[1]),
            "t_min": float(t_range[0]), "t_max": float(t_range[1]),
        }
        st.session_state["answers_sig"] = ans_sig
    except Exception:
        pass

# ---------- Data preview ----------
with tab_data:
    show = df.copy()
    show["avg_travel_min_eff"] = pd.to_numeric(show["avg_travel_min_eff"], errors="coerce")
    cols = ["GEMEINDE_NAME","vacancy_pct","avg_travel_min_eff","preference_score"]
    cols = [c for c in cols if c in show.columns]
    st.dataframe(show[cols].head(30).rename(columns={"avg_travel_min_eff":"avg_travel_min"}), use_container_width=True)
    st.caption(f"Total rows: {len(show)}")

# ---------- Map ----------
with tab_map:
    if map_mode == "Housing only":
        metric_col = "vacancy_pct"; legend = "Vacancy rate (%)"; colors = ["red","yellow","green"]
        vals = pd.to_numeric(df["vacancy_pct"], errors="coerce")
        vmin = float(np.nanpercentile(vals, 1)) if vals.notna().any() else 0.0
        vmax = float(np.nanpercentile(vals, 99)) if vals.notna().any() else 1.0
        df_render = df.copy()

    elif map_mode == "Commute only":
        metric_col = "avg_travel_min_eff"; legend = "Commute time from origin (min)"; colors = ["green","yellow","red"]
        vals = pd.to_numeric(df["avg_travel_min_eff"], errors="coerce")
        vmin = 0.0
        vmax = float(np.nanpercentile(vals, 99)) if vals.notna().any() else 120.0
        df_render = df.dropna(subset=[metric_col]).copy()

    else:
        metric_col = "preference_score"; legend = "Preference score (0–100)"; colors = ["red","yellow","green"]

        if "manual_weights" in st.session_state:
            w0 = float(st.session_state["manual_weights"].get("w0", 0.0))
            a  = float(st.session_state["manual_weights"].get("w_v", 2.0))
            b  = float(st.session_state["manual_weights"].get("w_t", 2.0))
            v_lo, v_hi = float(st.session_state["manual_weights"].get("v_min", v_range[0])), float(st.session_state["manual_weights"].get("v_max", v_range[1]))
            t_lo, t_hi = float(st.session_state["manual_weights"].get("t_min", t_range[0])), float(st.session_state["manual_weights"].get("t_max", t_range[1]))
        else:
            w0, a, b = resolve_prior(meta, st.session_state)
            v_lo, v_hi = v_range; t_lo, t_hi = t_range

        v_all = norm01_clipped(pd.to_numeric(df["vacancy_pct"], errors="coerce").values, v_lo, v_hi)
        t_all = norm01_clipped(pd.to_numeric(df["avg_travel_min_eff"], errors="coerce").values, t_lo, t_hi)
        pen_t = commute_penalty_shape(t_all, penalty_k)
        util  = (w0 + a * v_all - b * pen_t)

        finite = np.isfinite(util)
        if finite.any():
            util = util - np.median(util[finite])

        df_render = df.copy()
        df_render["preference_score"] = 100.0 * (1.0 / (1.0 + np.exp(-util)))
        vmin, vmax = 0.0, 100.0
        df_render = df_render.dropna(subset=["preference_score"])

    # Folium renderer
    def render_folium(df_map):
        import folium, branca.colormap as bcm
        from streamlit_folium import st_folium
        import streamlit.components.v1 as components
        colormap = bcm.LinearColormap(colors=colors, vmin=vmin, vmax=vmax); colormap.caption = legend
        m = folium.Map(location=[46.8, 8.2], zoom_start=7, tiles="OpenStreetMap")

        if metric_col == "vacancy_pct":
            df_map["vacancy_round"] = pd.to_numeric(df_map["vacancy_pct"], errors="coerce").round(2)
            fields, aliases = ["GEMEINDE_NAME","vacancy_round"], ["Municipality","Vacancy (%)"]
        elif metric_col == "avg_travel_min_eff":
            df_map["time_round"] = pd.to_numeric(df_map["avg_travel_min_eff"], errors="coerce").round(1)
            fields, aliases = ["GEMEINDE_NAME","time_round"], ["Municipality","Commute (min)"]
        else:
            df_map["score_round"] = pd.to_numeric(df_map["preference_score"], errors="coerce").round(1)
            df_map["vacancy_round"] = pd.to_numeric(df_map["vacancy_pct"], errors="coerce").round(2)
            df_map["time_round"] = pd.to_numeric(df_map["avg_travel_min_eff"], errors="coerce").round(1)
            fields, aliases = ["GEMEINDE_NAME","score_round","vacancy_round","time_round"], ["Municipality","Score","Vacancy (%)","Commute (min)"]

        folium.GeoJson(
            data=df_map.to_json(),
            style_function=lambda feat: {
                "fillColor": colormap(float(feat["properties"].get(metric_col))) if feat["properties"].get(metric_col) is not None else "#cccccc",
                "color": "white", "weight": 0.2, "fillOpacity": 0.85
            },
            tooltip=folium.features.GeoJsonTooltip(fields=fields, aliases=aliases, localize=True)
        ).add_to(m)
        colormap.add_to(m)

        try:
            st_folium(m, height=720, width=None)
        except Exception:
            html = m._repr_html_()
            components.html(html, height=720, scrolling=False)

    if df_render.empty:
        st.warning("No municipalities available to render for this selection.")
    else:
        render_folium(df_render)
'''

with open(APP_PATH, "w", encoding="utf-8") as f:
    f.write(app_code)

print("✅ Wrote app.py (robust scenarios + origin=0 + zero-shift)")



✅ Wrote app.py (robust scenarios + origin=0 + zero-shift)


In [21]:
# --- Launch Streamlit with Cloudflare tunnel ---
import os, re, time, subprocess

APP_DIR = "/content/streamlit_app"
APP_PATH = os.path.join(APP_DIR, "app.py")
CF_BIN  = "/content/cloudflared"
CF_LOG  = "/content/cflog.txt"
ST_LOG  = "/content/streamlit.log"

# Deps
!pip -q install streamlit streamlit-folium geopandas folium branca shapely pyproj rtree pydeck >/dev/null

# Kill stale procs
!pkill -f "cloudflared" || true
!pkill -f "streamlit run" || true

# Start Streamlit
st_out = open(ST_LOG, "w")
st_proc = subprocess.Popen(
    ["streamlit", "run", APP_PATH, "--server.port=8501", "--server.headless=true"],
    cwd=APP_DIR, stdout=st_out, stderr=subprocess.STDOUT, text=True
)

# Wait for healthz
def wait_for_streamlit(timeout=45):
    start = time.time()
    while time.time() - start < timeout:
        ok = subprocess.run(["curl", "-sS", "-m", "2", "http://127.0.0.1:8501/healthz"],
                            capture_output=True, text=True)
        if ok.returncode == 0 and "ok" in ok.stdout.lower():
            return True
        time.sleep(1)
    return False

print("⏳ Waiting for Streamlit...")
if not wait_for_streamlit():
    print("--- streamlit.log tail ---")
    !tail -n 80 "/content/streamlit.log"
    raise RuntimeError("Streamlit didn't become ready.")

print("✅ Streamlit up at http://127.0.0.1:8501")

# Cloudflare tunnel
!curl -sL -o "{CF_BIN}" https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x "{CF_BIN}"

def start_tunnel():
    subprocess.Popen([CF_BIN, "tunnel", "--url", "http://127.0.0.1:8501", "--no-autoupdate"],
                     stdout=open(CF_LOG, "w"), stderr=subprocess.STDOUT, text=True)

def find_url(deadline=25):
    pat = re.compile(r"https://[-a-z0-9]+\.trycloudflare\.com")
    start = time.time()
    while time.time() - start < deadline:
        try:
            with open(CF_LOG, "r", encoding="utf-8", errors="ignore") as f:
                log = f.read()
            m = pat.search(log)
            if m:
                return m.group(0)
        except Exception:
            pass
        time.sleep(1)
    return None

public_url = None
for attempt in range(1, 3):
    !pkill -f "cloudflared" || true
    start_tunnel()
    time.sleep(2)
    url = find_url(25)
    if url:
        public_url = url
        break
    print(f"⚠️  Tunnel attempt {attempt} failed; retrying...")

if not public_url:
    print("--- cloudflared log tail ---")
    !tail -n 120 "{CF_LOG}"
    raise RuntimeError("Cloudflare tunnel did not attach. Re-run this cell.")

print("\n🌐 Your Streamlit app is live at:", public_url)
print("\n--- streamlit.log tail ---")
!tail -n 60 "/content/streamlit.log"


^C
^C
⏳ Waiting for Streamlit...
✅ Streamlit up at http://127.0.0.1:8501
^C
⚠️  Tunnel attempt 1 failed; retrying...
^C

🌐 Your Streamlit app is live at: https://venues-exploration-estimates-programmers.trycloudflare.com

--- streamlit.log tail ---



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.41.110.118:8501



In [22]:
# ===== 13) Stop Streamlit & Cloudflare =====
!pkill -f "cloudflared" || true
!pkill -f "streamlit run" || true
print("Stopped Streamlit and Cloudflare tunnel.")


^C
^C
Stopped Streamlit and Cloudflare tunnel.
